In [8]:
from pyspark.sql.types import *

* 可以用schema初始化数据类型，但是还需要对应位置，就神烦，不整了，GTMD
* 如果用这个方法，需要在读取数据的时候增加参数schema，这里设置的结构就叫schema，所以直接写schema=schema
* 还需要导入包：`from pyspark.sql.types import *`
* schema写法：`schema = StructType([StructField("id", IntegerType()),StructField("title", StringType()),StructField("tagline", StringType()),StructField("vote_average", FloatType()),StructField("vote_count", IntegerType())])`
* 我觉得可以在python里直接处理好数据再读到这里

* 当你读取csv格式错误的时候，你就加一个`multiLine=True,escape='"'`，专治各种不服

In [9]:
df1 = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/movies_metadata.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [10]:
# 一眼就能看出来这个具体结构一眼看不出来，那就不看了吧
df1.show()

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+-----------

In [11]:
# 看一看这个数据包含哪些特征
df1.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- runtime: double (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: boolean (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nu

In [12]:
# 抽出我们需要的那些特征
bmd = df1.select('genres','id','tagline','title','vote_average','vote_count')
bmd.show()

+--------------------+-----+--------------------+--------------------+------------+----------+
|              genres|   id|             tagline|               title|vote_average|vote_count|
+--------------------+-----+--------------------+--------------------+------------+----------+
|[{'id': 16, 'name...|  862|                null|           Toy Story|         7.7|      5415|
|[{'id': 12, 'name...| 8844|Roll the dice and...|             Jumanji|         6.9|      2413|
|[{'id': 10749, 'n...|15602|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|[{'id': 35, 'name...|31357|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|[{'id': 35, 'name...|11862|Just When His Wor...|Father of the Bri...|         5.7|       173|
|[{'id': 28, 'name...|  949|A Los Angeles Cri...|                Heat|         7.7|      1886|
|[{'id': 35, 'name...|11860|You are cordially...|             Sabrina|         6.2|       141|
|[{'id': 28, 'name...|45325|The Original Bad ...| 

bmd的数据类型:

`DataFrame[genres: string, id: string, tagline: string, title: string, vote_average: double, vote_count: int]`

* 知道怎么转换数据类型了，开心！在select里面写`表名['列名'].astype('int')`

In [13]:
bmd2 = bmd.select('genres',bmd['id'].astype('int'),'tagline','title','vote_average','vote_count')

bmd2的数据类型:

`DataFrame[genres: string, id: int, tagline: string, title: string, vote_average: double, vote_count: int]`

In [14]:
cres = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/credits.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [15]:
cres.show()

+--------------------+--------------------+-----+
|                cast|                crew|   id|
+--------------------+--------------------+-----+
|[{'cast_id': 14, ...|[{'credit_id': '5...|  862|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 8844|
|[{'cast_id': 2, '...|[{'credit_id': '5...|15602|
|[{'cast_id': 1, '...|[{'credit_id': '5...|31357|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11862|
|[{'cast_id': 25, ...|[{'credit_id': '5...|  949|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11860|
|[{'cast_id': 2, '...|[{'credit_id': '5...|45325|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9091|
|[{'cast_id': 1, '...|[{'credit_id': '5...|  710|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9087|
|[{'cast_id': 9, '...|[{'credit_id': '5...|12110|
|[{'cast_id': 1, '...|[{'credit_id': '5...|21032|
|[{'cast_id': 1, '...|[{'credit_id': '5...|10858|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 1408|
|[{'cast_id': 4, '...|[{'credit_id': '5...|  524|
|[{'cast_id': 6, '...|[{'credit_id': '5...| 4584|


cres的数据类型：

`DataFrame[cast: string, crew: string, id: int]`

In [16]:
kws = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/keywords.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [17]:
kws.show()

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|[{'id': 931, 'nam...|
| 8844|[{'id': 10090, 'n...|
|15602|[{'id': 1495, 'na...|
|31357|[{'id': 818, 'nam...|
|11862|[{'id': 1009, 'na...|
|  949|[{'id': 642, 'nam...|
|11860|[{'id': 90, 'name...|
|45325|                  []|
| 9091|[{'id': 949, 'nam...|
|  710|[{'id': 701, 'nam...|
| 9087|[{'id': 833, 'nam...|
|12110|[{'id': 3633, 'na...|
|21032|[{'id': 1994, 'na...|
|10858|[{'id': 840, 'nam...|
| 1408|[{'id': 911, 'nam...|
|  524|[{'id': 383, 'nam...|
| 4584|[{'id': 420, 'nam...|
|    5|[{'id': 612, 'nam...|
| 9273|[{'id': 409, 'nam...|
|11517|[{'id': 380, 'nam...|
+-----+--------------------+
only showing top 20 rows



kws的数据类型：

`DataFrame[id: int, keywords: string]`

In [18]:
df2 = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/ratings.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [19]:
rats = df2.select(df2['userId'].alias('id'),'movieID','rating')
rats.show()

+---+-------+------+
| id|movieID|rating|
+---+-------+------+
|  1|    110|   1.0|
|  1|    147|   4.5|
|  1|    858|   5.0|
|  1|   1221|   5.0|
|  1|   1246|   5.0|
|  1|   1968|   4.0|
|  1|   2762|   4.5|
|  1|   2918|   5.0|
|  1|   2959|   4.0|
|  1|   4226|   4.0|
|  1|   4878|   5.0|
|  1|   5577|   5.0|
|  1|  33794|   4.0|
|  1|  54503|   3.5|
|  1|  58559|   4.0|
|  1|  59315|   5.0|
|  1|  68358|   5.0|
|  1|  69844|   5.0|
|  1|  73017|   5.0|
|  1|  81834|   5.0|
+---+-------+------+
only showing top 20 rows



In [20]:
rats.cache()

DataFrame[id: int, movieID: int, rating: double]

rats的数据类型：

`DataFrame[id: int, movieID: int, rating: double]`

### 到这个部分，所有的数据都读取进来，完成了清理，并转换成了需要的格式！